#### Text Summary
So now lets build our model for our data but here we will not create model we will just train the existing model

In [34]:
# !pip install datasets
# !pip install transformers
# ! pip install sentencepiece
# !pip install evaluate rouge_score
# ! pip install transformers[torch]
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [2]:
#import

from datasets import load_dataset


here we will use dataset to train a model and use it for give me summary of some test data

In [3]:
dataset_name ='amazon_reviews_multi'
language = 'en'
dataset = load_dataset(dataset_name, language)

In this dataset we have 3 sets of data train, test and validation.
To be more on experimental side i will use test data as training data while training and validation data as test data as data size is too big and take hours on colab

In [4]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})

Let's see what's inside the each reivew

In [5]:
dataset['train'][10]

{'review_id': 'en_0210507',
 'product_id': 'product_en_0724267',
 'reviewer_id': 'reviewer_en_0031791',
 'stars': 1,
 'review_body': 'I am disappointed in this purchase. I bought one of these in another color and in size XL and it fit great. This one I bought in white in XL and it fits like a Medium. I cannot wear this top and am really disappointed about that, It is advertised as XL but seriously this top would only fit someone who would be a size s/m. An XXL would have been a better choice for me because the sizing is incorrect on this product.',
 'review_title': 'Not what I ordered',
 'language': 'en',
 'product_category': 'apparel'}

There are reivew_id, product_id, reviewer_id, stars, review_body, review_title, language, product_category but for our case we will use only reivew_title as summary and review_body as the text

#### now lets look at data its optional as i am choosing the small dataset from this big data so that training and testing will be fast for me on a specific cateogory

In [6]:
import pandas as pd
import numpy as np

In [7]:
df = pd.DataFrame(dataset['train'])

So maximum reviews are of home category items
Now we will take out least no. of counts category for our training

In [8]:
df['product_category'].value_counts()

home                        17679
apparel                     15951
wireless                    15717
other                       13418
beauty                      12091
drugstore                   11730
kitchen                     10382
toy                          8745
sports                       8277
automotive                   7506
lawn_and_garden              7327
home_improvement             7136
pet_products                 7082
digital_ebook_purchase       6749
pc                           6401
electronics                  6186
office_product               5521
shoes                        5197
grocery                      4730
book                         3756
baby_product                 3150
furniture                    2984
jewelry                      2747
camera                       2139
industrial_supplies          1994
digital_video_download       1364
luggage                      1328
musical_instruments          1102
video_games                   775
watch         

In [9]:
df['product_category'].value_counts()[-1:] # see least no. of review

personal_care_appliances    75
Name: product_category, dtype: int64

So i will take personal care appliances for now and to train fast and lets delete what we are not using to save memory

In [10]:
del df

Create a function to get personal_care_applianaces from the dataset

In [7]:
def filter_category(data):
  return (data['product_category'] == 'personal_care_appliances')

In [8]:
filter_data = dataset.filter(filter_category) # here filter is a function that is there on datasets to filter data and use parallel processing

In [9]:
filter_data

DatasetDict({
    train: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 75
    })
    validation: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 1
    })
    test: Dataset({
        features: ['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 2
    })
})

Now as you can see we have a very small data set that we will use for our case to train.

Next we will get a Tokenizer and Model to train with pre trained

In [10]:
# lets delete full dataset
del dataset

In [11]:
from transformers import AutoTokenizer

In [12]:
model_name = 'google/mt5-small' # he mt 5 is 5th version of mt model and small is its size
tokenizer = AutoTokenizer.from_pretrained(model_name) # based on name of model it gets tokenizer of it

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Cool, now we have tokenizer to convert text to tokens or in our case reviews to tokens

In [13]:
def preprocess_function(data):
  """This function is used to fix things tokenizer like max input length that can be given,
      maximum target length if maximum size exceed the length then truncate that data and lastly
      we add labels to model inputs data
  """
  max_input_length = 512 #
  max_target_length = 30
  model_inputs = tokenizer(data['review_body'], max_length=max_input_length, truncation=True)
  labels = tokenizer(data['review_title'], max_length=max_target_length, truncation=True)
  model_inputs['labels'] = labels['input_ids']
  return model_inputs


In [14]:
tokenize_data = filter_data.map(preprocess_function, batched=True) # here map isfunction that on datasets

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Now lets built the model first and then evaluate functions to evaluate the data

In [15]:
from transformers import AutoModelForSeq2SeqLM

In [16]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

Now this model need evaluation criteria, training arg and train and test data so lets do

frist: evaluate function

In [17]:
import evaluate

In [18]:
rouge_score = evaluate.load('rouge')

In [19]:
# Now we have rouge score to evaluate and its time to make argument object
# for that first need to import arguments
from transformers import Seq2SeqTrainingArguments

In [20]:
# now we will create some details about it
batch_size = 8
num_train_epochs = 8
logging_steps = len(filter_data['train']) // batch_size
only_model_name = model_name.split('/')[-1] # to get model name not google

In [21]:
# here we will add arguments
args = Seq2SeqTrainingArguments(
    output_dir = '{only_model_name}-finetuned-amazon-en', # can give any name
    evaluation_strategy = 'epoch',
    learning_rate=5.6e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size= batch_size,
    weight_decay = 0.01,
    save_total_limit = 1,
    num_train_epochs= num_train_epochs,
    predict_with_generate=True,
    logging_steps = logging_steps
)

Now lets create how we will calculate compute metric to know the accuracy of model

In [22]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  # decode the prediction
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  # gets the labels
  labels = np.where(labels!= -100, labels, tokenizer.pad_token_id)
  # deocde the labels
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
  # get prediction
  decoded_preds = ['\n'.join(sent_tokenize(pred.strip())) for pred in decoded_preds]
  # get labels
  decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
  # calculate rouge score
  result = rouge_score.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
  return result

Last step before give this data to model with args and compute metrics is to get data collator

In [23]:
from transformers import DataCollatorForSeq2Seq

In [24]:
# here to data collator we need to give tokenizer and model details
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [25]:
# from tokenizer_data lets remove unwanted data to save memory
tokenized_datasets = tokenize_data.remove_columns(
    filter_data['train'].column_names
)

In [26]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 75
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2
    })
})

Like we had pipeline here also we have a Seq2SeqTrainer that will have args, data, data collator, tokenizers, compute_metrics and give us result

In [31]:
from transformers import Seq2SeqTrainer
from nltk.tokenize import sent_tokenize # to create sentence of data

In [35]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset = tokenized_datasets['validation'],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics
)

In [36]:
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,20.867100,14.306795,0.000000,0.000000,0.000000,0.000000
2,22.805800,16.217054,0.000000,0.000000,0.000000,0.000000
3,21.997500,15.378480,0.000000,0.000000,0.000000,0.000000
4,20.580900,14.806763,0.000000,0.000000,0.000000,0.000000
5,18.926700,14.403424,0.000000,0.000000,0.000000,0.000000
6,20.218700,14.803487,0.000000,0.000000,0.000000,0.000000
7,19.901300,14.893433,0.000000,0.000000,0.000000,0.000000
8,18.977500,15.239052,0.000000,0.000000,0.000000,0.000000


TrainOutput(global_step=80, training_loss=20.290340995788576, metrics={'train_runtime': 863.2464, 'train_samples_per_second': 0.695, 'train_steps_per_second': 0.093, 'total_flos': 93201427445760.0, 'train_loss': 20.290340995788576, 'epoch': 8.0})

In [37]:
trainer.evaluate()

{'eval_loss': 15.239051818847656,
 'eval_rouge1': 0.0,
 'eval_rouge2': 0.0,
 'eval_rougeL': 0.0,
 'eval_rougeLsum': 0.0,
 'eval_runtime': 0.6349,
 'eval_samples_per_second': 1.575,
 'eval_steps_per_second': 1.575,
 'epoch': 8.0}

We got a good accracy on a small dataset.
Great!